# Exploration

In [6]:
import csv
import re
from collections import Counter
from itertools import islice
from tqdm import tqdm_notebook
from fog.key import fingerprint

In [42]:
# Params
DATA = '../data/didascalies.csv'

GROUPS = [
    'larem',
    'rem',
    'modem',
    'lr',
    'lfi',
    'fi',
    'lc',
    'udi',
    'ng',
    'gdr',
    'mda'
]

GROUPS = [(g, re.compile('\\b' + g + '\\b')) for g in GROUPS]

BLACK_LIST = [
    'article',
    'motion',
    'séance', # Possibilité d'étudier les suspensions via ce mot-clé
    'amendement',
    'projet',
    'scrutin'
]

EXCLAMATION_RE = re.compile('«([^»]+)»')

In [59]:
# Helpers
TAGS_RE = re.compile(r'<[^>]+>')

def strip_tags(html):
    return re.sub(TAGS_RE, '', html)

def is_didascalie(line):
    if line['nom'] != 'NULL' or line['parlementaire'] != 'NULL':
        return False
    
    raw_intervention = fingerprint(strip_tags(line['intervention']))
    
    if 'cohesion' in raw_intervention:
        print(fingerprint(line['titre']), raw_intervention)
    if fingerprint(line['titre']) == raw_intervention:
        return False
    
    return True

def is_relevant(intervention):
    intervention = intervention.lower()
    
    return (
        'applaud' in intervention or
        'murmur' in intervention or
        'rire' in intervention or
        'banc' in intervention
    )

def extract_groups(intervention):
    intervention = intervention.lower()
    
    if 'bancs' not in intervention:
        return None
    
    groups = [g for g, pattern in GROUPS if pattern.search(intervention)]
    
    return groups

In [60]:
COUNTER = Counter()
EXCLAMATIONS = Counter()
NB_DASHES = 0
with open(DATA, 'r') as f:
    reader = csv.DictReader(f)
    
    for line in islice(reader, None):
        if not is_didascalie(line):
            continue
            
        key = strip_tags(line['intervention']).lower()
        
        drop = False
        
        for b in BLACK_LIST:
            if b in key:
                drop = True
                break
        
        if not drop:
            COUNTER[key[0:10]] += 1
            
            if '–' in line['intervention'] or '- ' in line['intervention']:
                # print(line['intervention'])
                NB_DASHES += 1
                
            for m in re.finditer(EXCLAMATION_RE, line['intervention']):
                EXCLAMATIONS[m.group(0).lower().strip()] += 1

print(NB_DASHES)

cohesion des territoires cohesion des territoires
b cohesion des etat mission territoires << >> b cohesion des etat mission territoires
b cohesion des etat mission territoires << >> b cohesion des etat mission territoires
b cohesion des etat mission territoires << >> b cohesion des etat mission territoires
b cohesion des etat mission territoires << >> adoptes cohesion credits de des la les mission modifies sont territoires
cohesion des territoires cohesion des territoires
agence cohesion creation des dune la nationale pour territoires agence cohesion creation des dune la nationale pour territoires
1141


In [36]:
# TODO: handle "même mouvement"
# TODO: handle exclamations

COUNTER.most_common()

[('applaudiss', 13885),
 ('exclamatio', 2709),
 ('sourires. ', 1484),
 ('nombre de ', 987),
 ('mêmes mouv', 498),
 ('protestati', 359),
 ('rappel au ', 257),
 ('« ah ! » s', 210),
 ('sourires', 199),
 ('vifs appla', 158),
 ('sourires.', 119),
 ('rires. ', 119),
 ('rires et a', 115),
 ('mmes et mm', 96),
 ('rires sur ', 91),
 ('sourires e', 90),
 ('sourires s', 87),
 ('« eh oui !', 84),
 ('rires et e', 79),
 ('murmures s', 76),
 ('vives excl', 75),
 ('« bravo ! ', 66),
 ('rappels au', 64),
 ('les crédit', 51),
 ('« oh ! » s', 46),
 ('« très bie', 45),
 ('« non ! » ', 38),
 ('seconde pa', 38),
 ('mission « ', 37),
 ('nouvelles ', 35),
 ('vives prot', 34),
 ('la proposi', 31),
 ('« oui ! » ', 30),
 ('évolution ', 24),
 ('approbatio', 19),
 ('« ah ! » e', 18),
 ('rires', 18),
 ('murmures. ', 16),
 ('à dix-sept', 16),
 ('immigratio', 16),
 ('les député', 14),
 ('m. jean la', 14),
 ('mme sylvia', 14),
 ('« si ! » s', 13),
 ('nouveaux a', 13),
 ("« c'est fa", 12),
 ('mouvements', 11),
 ('même

In [54]:
EXCLAMATIONS.most_common()

[('« ah ! »', 247),
 ('« eh oui ! »', 87),
 ('« bravo ! »', 82),
 ('« oh ! »', 55),
 ('« très bien ! »', 45),
 ('« non ! »', 42),
 ('« oui ! »', 31),
 ('« si ! »', 17),
 ("« c'est faux ! »", 16),
 ('« très juste ! »', 9),
 ('« chut ! »', 9),
 ("« c'est vrai ! »", 9),
 ('« exactement ! »', 8),
 ('« mais non ! »', 7),
 ('« ah ? »', 6),
 ('« et alors ? »', 6),
 ('« allô ! »', 5),
 ('« ah bon ? »', 4),
 ('« bien sûr ! »', 4),
 ('« en effet ! »', 4),
 ('« la question ! »', 4),
 ('« cohésion des territoires »', 4),
 ('« stop ! »', 4),
 ('« mais si ! »', 3),
 ('« tout à fait ! »', 3),
 ('« ah ! enfin ! »', 3),
 ('« rien ! »', 3),
 ('« mais oui ! »', 3),
 ('« excellent ! »', 3),
 ('« on ne comprend rien ! »', 3),
 ("« ce n'est pas la question ! »", 3),
 ('« merci ! »', 3),
 ('« immigration, asile et intégration »', 3),
 ('« culture »', 3),
 ('« médias, livre et industries culturelles »', 3),
 ('« et de lyon ! »', 3),
 ('« scandaleux ! »', 3),
 ('« il a raison ! »', 3),
 ('« une ministre ! »', 